<a href="https://colab.research.google.com/github.com/aristring/OptimizacionHiperparametros/blob/main/datos.csv" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Escuela de Ciencias Físicas y Matemática</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Optmización de Hiperparámetros
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Ariana Cabrera &bull; 2024-02
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

A lo largo de este taller, aplicaremos optimización de hiperparámetros en un modelo que elijas.

Los paquetes necesarios son:

In [80]:
# Paquetes necesarios
import pandas as pd  # Manejo de datos
import matplotlib.pyplot as plt  # Visualización

from sklearn.model_selection import train_test_split # División de datos
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Métrica de evaluación

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold  # Búsqueda de hiperparámetros
from sklearn.ensemble import RandomForestClassifier

import joblib

---
## <font color='264CC7'> Clasificación </font>


### <font color='264CC7'> Preprocesamiento de datos </font>

Primero necesitas el conjunto de datos. Los datos a utilzar son los seleccionados en la clase anterior.

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Carga el conjunto de datos y procésalos:
<ul>
  <li>Muestra algunos datos.</li>
  <li>Muestra una descripción de los datos.</li>
  <li>Escala los datos si es necesario.</li>
</ul>
</div>

In [62]:
# Leer los datos
raw = pd.read_csv('https://raw.githubusercontent.com/aristring/OptimizacionHiperparametros/refs/heads/main/datos.csv')

# Seleccionar columnas de interés (sin 'Rank')
numeric_cols = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Global_Sales']
class_col = ['Publisher']  # La columna objetivo será 'Genre'
data = raw[['Name', 'Genre', 'Platform', *numeric_cols, *class_col]]

# Seleccionar solo los primeros 300 registros
data = data.head(300)

Revisamos los datos

In [63]:
data.describe()

,NA_Sales,EU_Sales,JP_Sales,Global_Sales
count,300.000000,300.000000,300.000000,300.000000
mean,4.008133,2.471300,1.087733,8.343333
std,4.090170,2.475031,1.540248,7.282351
min,0.000000,0.000000,0.000000,4.010000
25%,2.172500,1.240000,0.060000,4.695000
50%,2.980000,1.930000,0.295000,5.835000
75%,4.280000,2.835000,1.572500,8.885000
max,41.490000,29.020000,10.220000,82.740000


Mostramos algunos datos

In [64]:
# Mostrar las primeras filas del dataset
display(data.head())

,Name,Genre,Platform,NA_Sales,EU_Sales,JP_Sales,Global_Sales,Publisher
0,Wii Sports,Sports,Wii,41.49,29.02,3.77,82.74,Nintendo
1,Super Mario Bros.,Platform,NES,29.08,3.58,6.81,40.24,Nintendo
2,Mario Kart Wii,Racing,Wii,15.85,12.88,3.79,35.82,Nintendo
3,Wii Sports Resort,Sports,Wii,15.75,11.01,3.28,33.00,Nintendo
4,Pokemon Red/Pokemon Blue,Role-Playing,GB,11.27,8.89,10.22,31.37,Nintendo


In [67]:
# Identificar los publishers con más de un registro
publisher_counts = data['Publisher'].value_counts()
valid_publishers = publisher_counts[publisher_counts > 1].index

# Filtrar los datos para mantener solo los publishers con más de un registro
filtered_data = data[data['Publisher'].isin(valid_publishers)]

# Seleccionar las columnas numéricas excluyendo 'Rank'
numeric_cols = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Global_Sales']
X = filtered_data[numeric_cols]
y = filtered_data['Publisher']

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Verificar las dimensiones de los conjuntos resultantes
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((235, 4), (59, 4), (235,), (59,))


### <font color='264CC7'> Modelo </font>


<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Selecciona el mejor modelo de las clases anteriores.
<ul>
  <li>Muestra los hiperparámetros del modelo.</li>
  <li>Consulta qué significan al menos 4 hiperparámetros.</li>
  <li>Selecciona los hiperparámetros que deseas optimizar, al menos 3.</li>
</ul>
</div>

In [69]:
# Seleccionar columnas numéricas para el modelo
X = filtered_data.drop(columns=['Name', 'Genre', 'Platform', 'Publisher'], errors='ignore')  # 'Publisher' es la columna objetivo
y = filtered_data['Publisher']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=28, stratify=y)

# Crear el modelo base RandomForestClassifier
modelo_base = RandomForestClassifier(random_state=28)

# Entrenar el modelo
modelo_base.fit(X_train, y_train)

# Mostrar los hiperparámetros del modelo base
modelo_base_params = modelo_base.get_params()
modelo_base_params


{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 28,
 'verbose': 0,
 'warm_start': False}

**Hiperparámetros del modelo base**

A continuación, se describen algunos de los hiperparámetros más importantes del modelo `RandomForestClassifier`:

- **`bootstrap`**: Determina si se utiliza o no el muestreo bootstrap para construir los árboles. Por defecto es `True`.
- **`ccp_alpha`**: Parámetro de complejidad para la poda mínima de los árboles. El valor predeterminado es `0.0`, lo que significa que no se realiza poda.
- **`criterion`**: Define la función utilizada para medir la calidad de las divisiones en los nodos. Por defecto es `'gini'` (índice de Gini).
- **`max_depth`**: La profundidad máxima de los árboles. Si es `None`, los árboles se expanden hasta que todas las hojas sean puras o contengan menos de `min_samples_split` muestras.
- **`max_features`**: El número de características a considerar al buscar la mejor división. Por defecto es `'sqrt'` (la raíz cuadrada del número total de características).
- **`n_estimators`**: Número de árboles en el bosque. El valor predeterminado es `100`.

Con esto seleccionaré `n_estimators`, `min_samples_split` y `max_depth`.

### <font color='264CC7'> Optimización por GridSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica GridSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 3 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [70]:
from sklearn.model_selection import GridSearchCV, KFold

# Definir los hiperparámetros a optimizar
parametros = {
    'n_estimators': [50, 75, 100],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
}

# Configurar validación cruzada con 5 particiones
k_fold = KFold(n_splits=5, shuffle=True, random_state=28)

# Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=modelo_base,
    param_grid=parametros,
    cv=k_fold,
    scoring='accuracy',
    n_jobs=-1  # Paralelización
)

# Entrenar el modelo con la búsqueda en malla
grid_search.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=28, shuffle=True),
             estimator=RandomForestClassifier(random_state=28), n_jobs=-1,
             param_grid={'max_depth': [5, 10, 15],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 75, 100]},
             scoring='accuracy')

In [71]:
# Obtener los mejores parámetros y el puntaje
mejores_parametros = grid_search.best_params_
mejor_score = grid_search.best_score_

mejores_parametros, mejor_score

({'max_depth': 5, 'min_samples_split': 10, 'n_estimators': 100},
 0.47659574468085103)

In [75]:
# Realizar predicciones usando el mejor modelo encontrado
y_pred = grid_search.best_estimator_.predict(X_test)

# Precisión del modelo con dos decimales
accuracy = round(accuracy_score(y_test, y_pred), 2)
print("Precisión del modelo:", accuracy)

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(cm)

# Reporte de clasificación con zero_division=0
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred, zero_division=0))

Precisión del modelo: 0.63
Matriz de confusión:
[[ 6  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  6  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  3  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1 19  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  2  0  3  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0]]
Reporte de clasificación:
                              precision    recall  f1-score   support

                  Activision       0.4

**Análisis de Resultados**

1. Parámetros Óptimos y Puntaje
Después de realizar la optimización de hiperparámetros con `GridSearchCV`, se obtuvieron los siguientes parámetros óptimos:
- **max_depth**: 5
- **min_samples_split**: 10
- **n_estimators**: 100

El puntaje promedio durante la validación cruzada fue:
- **Accuracy**: 0.4766 (47.66%).

2. Evaluación del Modelo
La precisión del modelo evaluado en el conjunto de prueba fue:
- **Precisión**: 0.63 (63%).

**Matriz de Confusión**

La matriz de confusión muestra cómo se clasificaron correctamente y incorrectamente las muestras. Algunos puntos clave:
- **Nintendo** (clase mayoritaria) fue clasificado correctamente en 19 de 20 casos.
- Varias clases como **Atari**, **Capcom**, **Sega**, entre otras, no tuvieron predicciones correctas.



### <font color='264CC7'> Optimización por RandomSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica RandomSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 5 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Usa RandomSearchCV con 25 iteraciones.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [76]:
# Definir los hiperparámetros a optimizar con al menos 5 valores para cada uno
parametros = {
    'n_estimators': [50, 75, 100, 125, 150],
    'max_depth': [3, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 6, 8],
}

# Configurar validación cruzada con 5 particiones
k_fold = KFold(n_splits=5, shuffle=True, random_state=28)

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=28),
    param_distributions=parametros,
    n_iter=25,  # 25 iteraciones
    cv=k_fold,
    scoring='accuracy',
    random_state=28,
    n_jobs=-1  # Paralelización
)

In [77]:
# Entrenar el modelo con la búsqueda aleatoria
random_search.fit(X_train, y_train)

# Obtener los mejores parámetros y el puntaje
mejores_parametros = random_search.best_params_
mejor_score = random_search.best_score_

print("Mejores parámetros:", mejores_parametros)
print("Mejor puntaje (accuracy):", mejor_score)

Mejores parámetros: {'n_estimators': 50, 'min_samples_leaf': 4, 'max_depth': 20}
Mejor puntaje (accuracy): 0.48936170212765956


**Mejores Parámetros**
Después de realizar la optimización de hiperparámetros con `RandomizedSearchCV`, los parámetros óptimos encontrados fueron:
- **n_estimators**: 50 (número de árboles en el bosque).
- **min_samples_leaf**: 4 (mínimo número de muestras requeridas en una hoja de decisión).
- **max_depth**: 20 (profundidad máxima del árbol).

**Rendimiento del Modelo**
El mejor puntaje de precisión (**accuracy**) obtenido durante la validación cruzada fue:
- **Accuracy**: 0.4894 (48.94%).

**Observaciones**
- El puntaje obtenido indica que el modelo tiene un rendimiento limitado en las predicciones, probablemente debido al desequilibrio en las clases o la naturaleza de los datos.
- A pesar de ello, los parámetros ajustados optimizan el balance entre sobreajuste y subajuste, dado el uso de restricciones como `min_samples_leaf` y `max_depth`.


### <font color='264CC7'> Guardado de modelo </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Con los parámetros óptimos que mejor resultado dieron, reentrena el modelo, muestra su score y guárdalo.
</div>

In [78]:
# Reentrenar el modelo con los mejores parámetros
modelo_final = RandomForestClassifier(
    n_estimators=50,
    min_samples_leaf=4,
    max_depth=20,
    random_state=28
)

# Entrenar el modelo con el conjunto de entrenamiento
modelo_final.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
y_pred = modelo_final.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Precisión del modelo final:", round(accuracy, 2))

Precisión del modelo final: 0.61


Con **GridSearch**, el puntaje promedio durante la validación cruzada fue del **47.66%**, mientras que la precisión en el conjunto de prueba alcanzó el **63%**, lo que refleja una mejora significativa en la generalización. Por otro lado, con **RandomSearch**, el puntaje de validación cruzada fue ligeramente superior (**48.94%**), pero la precisión final en el conjunto de prueba fue menor, con un **61%**. Esto sugiere que ambos métodos lograron un rendimiento comparable, pero GridSearch mostró una mejor capacidad de generalización en este caso.

### <font color='264CC7'> Publicación </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Coloca el este cuaderno y el modelo en tu repositorio de GitHub. Agrega una licencia MIT y un README.md donde se explique el contenido del repositorio, los datos utilizados y los resultados obtenidos.
</div>